### Document search with embedding
這個範例示範如何使用Gemini API建立嵌入,以便於可以執行文件的搜尋.
您將使用 Python 用戶端程式庫建立一個詞嵌入，以便將搜尋字串或問題與文件內容進行比較。
在本教學中，您將使用嵌入對一組文件執行文件搜索，以提出與 Google Car 相關的問題。

### 重點
您將選擇一個模型。任何嵌入模型(embedding model)都適用於本教程，但對於實際應用來說，選擇一個特定的模型並堅持使用它非常重要。不同模型的輸出彼此不相容。

In [82]:
import os
import google.generativeai as genai
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
#檢查目前可以用的embedding model
for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


### 指定任務類型
| 任務類型 | 說明 |
|:--|:--|
| RETRIEVAL_QUERY | Specifies the given text is a query in a search/retrieval setting. |
| RETRIEVAL_DOCUMENT | Specifies the given text is a document in a search/retrieval setting. |
| SEMANTIC_SIMILARITY | Specifies the given text will be used for Semantic Textual Similarity (STS). |
| CLASSIFICATION | Specifies that the embeddings will be used for classification. |
| CLUSTERING | Specifies that the embeddings will be used for clustering. |

In [83]:
#Note: Specifying a title for RETRIEVAL_DOCUMENT provides better quality embeddings for retrieval.
#建立向量資料
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications")

model = 'models/text-embedding-004'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding['embedding'])

[-0.0021609084, -0.0031644478, -0.060120754, -0.0071218493, 0.0008775481, 0.040581927, 0.04457149, 0.03552469, -0.04746538, 0.008888608, -0.02795826, 0.011335696, -0.0024438712, 0.0030851837, -0.018796137, -0.055550937, 0.03142646, 0.0006549187, -0.11370059, 0.063708074, -0.021750016, -0.021367034, -0.09982074, -0.008604756, -0.03330058, -0.012815642, 0.07153146, 0.037064772, 0.022970116, 0.043331202, 0.010670608, 0.040685344, 0.036361404, -0.036222056, -0.01779936, -0.014820963, 0.0053205034, -0.017382707, 0.07044941, 0.002021254, -0.018208738, 0.01755808, 0.006493214, 0.12724239, -0.023805201, 0.010057808, -0.0006948924, 0.070856266, -0.05645728, 0.018311134, 0.090462275, 0.021575555, -0.06656089, 0.026865067, -0.0034812444, -0.0011228676, -0.06535635, -0.0018169138, 0.08672994, 0.028747609, -0.024817271, 0.004653876, -0.058998518, 0.032061696, -0.022604035, -0.015454275, -0.013758674, 0.021129983, -0.047893394, 0.02573244, 0.01302824, -0.018002003, -0.039879415, 0.023450835, -0.0042

In [84]:
DOCUMENT1 = {
    "title": "操作氣候控制系統",
    "content": "您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統，請使用中央控制台上的按鈕和旋鈕。 溫度：溫度旋鈕控制車內溫度。順時針轉動旋鈕可升高溫度，逆時針轉動旋鈕可降低溫度。氣流：氣流旋鈕控制車內氣流。順時針轉動旋鈕可增加氣流，逆時針轉動旋鈕可減少氣流。風扇速度：風扇速度旋鈕控制風扇的速度。順時針轉動旋鈕可增加風扇速度，逆時針轉動旋鈕可降低風扇速度。模式：模式按鈕可讓您選擇所需的模式。可用的模式有：自動：汽車將自動調節溫度和氣流以保持舒適的水平。涼爽：汽車會向車內吹出涼風。加熱：汽車會將暖風吹入車內。除霜：汽車將向擋風玻璃吹暖風除霜。"}
DOCUMENT2 = {
    "title": "觸控螢幕",
    "content": "您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。要使用觸控屏，只需觸摸所需的圖標。 例如，您可以觸摸「導航」圖示以取得前往目的地的路線，或觸摸「音樂」圖示播放您喜愛的歌曲。"}
DOCUMENT3 = {
    "title": "換擋",
    "content": "您的 Googlecar 配有自動變速箱。要換檔，只需將換檔桿移至所需位置。 停車：停車時使用此位置。車輪被鎖住，汽車無法移動。倒車：此位置用於倒車。空檔：當您在紅燈處或交通中停車時使用此位置。汽車沒有掛檔，除非你踩下油門，否則它不會移動。驅動：此位置用於向前行駛。低：此位置用於在雪地或其他濕滑條件下駕駛。"}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]
documents

[{'title': '操作氣候控制系統',
  'content': '您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統，請使用中央控制台上的按鈕和旋鈕。 溫度：溫度旋鈕控制車內溫度。順時針轉動旋鈕可升高溫度，逆時針轉動旋鈕可降低溫度。氣流：氣流旋鈕控制車內氣流。順時針轉動旋鈕可增加氣流，逆時針轉動旋鈕可減少氣流。風扇速度：風扇速度旋鈕控制風扇的速度。順時針轉動旋鈕可增加風扇速度，逆時針轉動旋鈕可降低風扇速度。模式：模式按鈕可讓您選擇所需的模式。可用的模式有：自動：汽車將自動調節溫度和氣流以保持舒適的水平。涼爽：汽車會向車內吹出涼風。加熱：汽車會將暖風吹入車內。除霜：汽車將向擋風玻璃吹暖風除霜。'},
 {'title': '觸控螢幕',
  'content': '您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。要使用觸控屏，只需觸摸所需的圖標。 例如，您可以觸摸「導航」圖示以取得前往目的地的路線，或觸摸「音樂」圖示播放您喜愛的歌曲。'},
 {'title': '換擋',
  'content': '您的 Googlecar 配有自動變速箱。要換檔，只需將換檔桿移至所需位置。 停車：停車時使用此位置。車輪被鎖住，汽車無法移動。倒車：此位置用於倒車。空檔：當您在紅燈處或交通中停車時使用此位置。汽車沒有掛檔，除非你踩下油門，否則它不會移動。驅動：此位置用於向前行駛。低：此位置用於在雪地或其他濕滑條件下駕駛。'}]

In [85]:
import pandas as pd
import numpy as np
import textwrap

df = pd.DataFrame(documents)
df.columns = ['Title','Text']
df

,Title,Text
0,操作氣候控制系統,您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統...
1,觸控螢幕,您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。...
2,換擋,您的 Googlecar 配有自動變速箱。要換檔，只需將換檔桿移至所需位置。 停車：停車時使...


In [86]:
def embed_fn(title, text):
    return genai.embed_content(model=model,
                              content=text,
                              task_type="retrieval_document",
                              title=title
                              )["embedding"]

df['Embeddings'] = df.apply(lambda row:embed_fn(row['Title'], row['Text']),axis=1)
df


,Title,Text,Embeddings
0,操作氣候控制系統,您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統...,"[0.021456055, 0.04029468, -0.053465027, 0.0147..."
1,觸控螢幕,您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。...,"[0.016744798, 0.032688692, -0.050210994, 0.025..."
2,換擋,您的 Googlecar 配有自動變速箱。要換檔，只需將換檔桿移至所需位置。 停車：停車時使...,"[0.024208821, 0.035073824, -0.055515066, 0.020..."


In [101]:
query = "如何在Google汽車上使用觸控螢幕？"
model = 'models/text-embedding-004'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")


In [88]:
import numpy as np
#numpy.stack() 還可以設定參數 axis，0 表示 row 軸 ( 預設值 )，1 為 column 軸，如果陣列形狀不同，合併會發生錯誤。

a = np.array([[1,1], [2,2], [3,3]]) # 3x2的陣列
b = np.array([[4,4], [5,5], [6,6]]) # 3x2的陣列 
c = np.stack((a,b), axis=0) #axis設定為0
print(c) #堆積成為2x3x2
d = np.stack((a, b), axis=1)
print(d) #堆積成為3x2x2


[[[1 1]
  [2 2]
  [3 3]]

 [[4 4]
  [5 5]
  [6 6]]]
[[[1 1]
  [4 4]]

 [[2 2]
  [5 5]]

 [[3 3]
  [6 6]]]


In [89]:
#np.dot()計算內積
import numpy as np
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([[1, 2, 3], [1, 2, 3], [1, 2, 3]])

np.dot(a,b)

array([[ 6, 12, 18],
       [15, 30, 45],
       [24, 48, 72]])

In [102]:

def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  print(dot_products)
  idx = np.argmax(dot_products)
  print(idx)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

In [103]:
passage = find_best_passage(query, df)
passage

[0.72597223 0.72970017 0.7303892 ]
2


'您的 Googlecar 配有自動變速箱。要換檔，只需將換檔桿移至所需位置。 停車：停車時使用此位置。車輪被鎖住，汽車無法移動。倒車：此位置用於倒車。空檔：當您在紅燈處或交通中停車時使用此位置。汽車沒有掛檔，除非你踩下油門，否則它不會移動。驅動：此位置用於向前行駛。低：此位置用於在雪地或其他濕滑條件下駕駛。'

In [105]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""您是一個樂於助人且資訊豐富的機器人，可以使用下面參考段落中的文字來回答問題。 \
 一定要用完整的句子來回答，內容全面，包括所有相關的背景資訊。 \
 但是，你正在與非技術受眾交談，因此請務必分解複雜的概念和\
 採取友好、交談的語氣。 \
 如果該段落與答案無關，你可以忽略它。\
 並直接告訴它你無法回答,不要有其它描述。
 QUESTION: '{query}'
 PASSAGE: '{relevant_passage}'

 ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [106]:
prompt = make_prompt(query, passage)
print(prompt)

您是一個樂於助人且資訊豐富的機器人，可以使用下面參考段落中的文字來回答問題。  一定要用完整的句子來回答，內容全面，包括所有相關的背景資訊。  但是，你正在與非技術受眾交談，因此請務必分解複雜的概念和 採取友好、交談的語氣。  如果該段落與答案無關，你可以忽略它。 並直接告訴它你無法回答,不要有其它描述。
 QUESTION: '如何在Google汽車上使用觸控螢幕？'
 PASSAGE: '您的 Googlecar 配有自動變速箱。要換檔，只需將換檔桿移至所需位置。 停車：停車時使用此位置。車輪被鎖住，汽車無法移動。倒車：此位置用於倒車。空檔：當您在紅燈處或交通中停車時使用此位置。汽車沒有掛檔，除非你踩下油門，否則它不會移動。驅動：此位置用於向前行駛。低：此位置用於在雪地或其他濕滑條件下駕駛。'

 ANSWER:



In [107]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [95]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')
answer = model.generate_content(prompt)

In [108]:
from IPython.display import Markdown
Markdown(answer.text)

Google 汽車配備自動變速箱，換檔方式非常簡單。排檔桿上有幾個不同的檔位，分別對應不同的駕駛情況。

* **P (停車)**：停車時使用這個檔位。它會鎖住車輪，讓車輛完全靜止。

* **R (倒車)**：顧名思義，倒車時使用這個檔位。

* **N (空檔)**：當你在紅綠燈或塞車時，可以使用空檔。這個檔位讓車輛處於非驅動狀態，除非你踩油門，否則車子不會移動。

* **D (前進)**：這是你平常開車前進時使用的檔位。

* **L (低速檔)**：這個檔位適合在雪地、泥濘路面或其他濕滑路況行駛時使用，可以提供更大的牽引力。


所以，換檔時你只需要將排檔桿移動到你需要的檔位即可。
